In [1]:
import sys
from pathlib import Path
from os import getcwd
sys.path.append(str(Path(getcwd()).parent))

from datetime import date, timedelta
import codecs
import pandas as pd

pd.set_option('display.max_columns', None)

from os.path import join

from bulletin import default_input, default_output
from bulletin.services.metabase import Metabase
from bulletin.casos_confirmados import CasosConfirmados
from bulletin.utils.static import Municipios
from bulletin.utils.normalize import normalize_hash, normalize_labels, normalize_text, date_hash, normalize_number
from bulletin.utils import static
from bulletin.utils.xls_ import fit_cols

from datetime import datetime


---------------------------------------------------------------

root: /home/vinnylg/projects/covid19datascience/bulletin
parent: /home/vinnylg/projects/covid19datascience
default_input: /home/vinnylg/projects/covid19datascience/input
default_output: /home/vinnylg/projects/covid19datascience/output

---------------------------------------------------------------



In [2]:
municipios = Municipios()
municipios['mun_resid'] = municipios['municipio'].apply(normalize_text)
municipios.loc[municipios['uf']!='PR','mun_resid'] = municipios.loc[municipios['uf']!='PR','municipio'].apply(normalize_text) + '/' + municipios['uf']

In [3]:
mb = Metabase()
diario = mb.download_query('old_diario')
diario

Cookie:'8d43f********-****-****-****-************5aece'

saved_queries: ['diario', 'old_diario', 'all_notifica'] 

tmp_copy:->	 /home/vinnylg/projects/covid19datascience/input/queries/old_diario_len.csv /home/vinnylg/projects/covid19datascience/input/queries/tmp/old_diario_len_d290520211224.csv

query_size: 70375

select ... limit 262144 offset 0
Success code 200
tmp_copy:->	 /home/vinnylg/projects/covid19datascience/input/queries/old_diario_o0.csv /home/vinnylg/projects/covid19datascience/input/queries/tmp/old_diario_o0_d290520211225.csv
Download finish, time elapsed: 0:00:03.782973
downloaded shape (70376, 22)

Download query time elepsed:00:00:18


'/home/vinnylg/projects/covid19datascience/input/queries/old_diario.csv'

In [4]:
date_parser = lambda x: pd.to_datetime(x, errors='coerce', format='%d/%m/%Y')

notifica = pd.read_csv(diario,
                       sep=',',
                       converters = {
                           'paciente': normalize_text,
                           'origem': lambda x: normalize_number(x,fill=0),
                           'nome_mae': normalize_text,
                           'idade': lambda x: normalize_number(x,fill=0),
                           'pais_residencia': lambda x: normalize_number(x,fill=0),
                           'uf_residencia': lambda x: normalize_number(x,fill=0),
                           'ibge_residencia': lambda x: normalize_number(x,fill=999999),
                           'uf_unidade_notifica': lambda x: normalize_number(x,fill=0),
                           'ibge_unidade_notifica': lambda x: normalize_number(x,fill=999999),
                           'criterio_classificacao': lambda x: normalize_number(x,fill=0),
                           'evolucao': lambda x: normalize_number(x,fill=3) }
                       )


notifica['data_cura_obito'] = notifica['data_cura_obito'].apply(date_parser)
notifica['data_liberacao'] = notifica['data_liberacao'].apply(date_parser)

notifica.loc[notifica['evolucao']==5, 'evolucao'] = 2
notifica.loc[~notifica['evolucao'].isin([1,2,3]),'evolucao'] = 3

notifica['hash'] = (notifica['paciente'].apply(normalize_hash) +
                  notifica['idade'].astype(str) +
                  notifica['ibge_residencia'].astype(str) )


notifica['hash_atend'] = (notifica['paciente'].apply(normalize_hash) +
                  notifica['idade'].astype(str) +
                  notifica['ibge_unidade_notifica'].astype(str) )

notifica['hash_less'] = ( notifica['paciente'].apply(normalize_hash) +
                        notifica['idade'].apply(lambda x: str(x - 1)) +
                        notifica['ibge_residencia'].astype(str) )

notifica['hash_more'] = ( notifica['paciente'].apply(normalize_hash) +
                        notifica['idade'].apply(lambda x: str(x + 1)) +
                        notifica['ibge_residencia'].astype(str) )

notifica['hash_diag'] = notifica['paciente'].apply(normalize_hash) + notifica['data_liberacao'].apply(date_hash)

notifica = pd.merge(notifica.rename(columns={'ibge_residencia':'ibge'}),municipios[['ibge','rs','mun_resid','uf','municipio']],on='ibge',how='left').rename(columns={'ibge':'ibge_residencia'})
notifica = pd.merge(notifica.rename(columns={'ibge_unidade_notifica':'ibge'}),municipios[['ibge','mun_resid']].rename(columns={'mun_resid':'mun_atend'}),on='ibge',how='left').rename(columns={'ibge':'ibge_unidade_notifica'})

notifica = notifica.loc[~(
    (notifica['data_liberacao'].isna()) |
    (notifica['sexo'].isna())
)]

notifica = notifica.loc[(
    (notifica['data_liberacao'] < pd.to_datetime(datetime.today())) &
    (notifica['data_liberacao'] > pd.to_datetime(datetime.strptime('12/03/2020','%d/%m/%Y')))
)]

notifica.groupby('evolucao')[['id']].count()

,id
evolucao,
1,19745
2,1186
3,26488


In [5]:
cc = CasosConfirmados()
cc.load()
casosct = cc.df
casosct.tail()

databases: ['cc_21_05_2021', 'cc_22_05_2021', 'cc_23_05_2021', 'cc_25_05_2021', 'cc_27_05_2021', 'cc_28_05_2021']
Load /home/vinnylg/projects/covid19datascience/bulletin/database/casos_confirmados/cc_28_05_2021.pkl
loading Casos Confirmados from pkl time elepsed:00:00:25


,identificacao,id_notifica,uf_resid,ibge_resid,ibge_atend,nome,sexo,idade,laboratorio,dt_diag,comunicacao,is,evolucao,data_evolucao,data_com_evolucao,hash,hash_atend,hash_less,hash_more,hash_diag
1084375,1176616,-1,PR,411820,999999,ZENILTON PIRES MODESTO,M,39,"COVID-19, Biologia Molecular",2021-05-17,2021-05-28,NaN,3.0,NaT,NaT,ZENILTONPIRESMODESTO39,ZENILTONPIRESMODESTO39,ZENILTONPIRESMODESTO38,ZENILTONPIRESMODESTO40,ZENILTONPIRESMODESTO17052021
1084376,1176617,-1,PR,410720,410720,ZEUNILDE AGUIAR ACERMAO,F,37,"COVID-19, Biologia Molecular",2021-05-27,2021-05-28,2021-05-17 00:00:00,3.0,NaT,NaT,ZEUNILDEAGUIARACERMAO37,ZEUNILDEAGUIARACERMAO37,ZEUNILDEAGUIARACERMAO36,ZEUNILDEAGUIARACERMAO38,ZEUNILDEAGUIARACERMAO27052021
1084377,1176618,-1,PR,411840,411840,ZILDA LOPES,F,48,"COVID-19, Biologia Molecular",2021-05-16,2021-05-28,2021-05-09 00:00:00,3.0,NaT,NaT,ZILDALOPES48,ZILDALOPES48,ZILDALOPES47,ZILDALOPES49,ZILDALOPES16052021
1084378,1176619,-1,PR,411840,411840,ZILDA MACHADO PAZZETTO DOS SANTOS,F,42,Teste Rápido Antígeno (Imunocromatografia),2021-05-27,2021-05-28,2021-05-24 00:00:00,3.0,NaT,NaT,ZILDAMACHADOPAZZETTODOSSANTOS42,ZILDAMACHADOPAZZETTODOSSANTOS42,ZILDAMACHADOPAZZETTODOSSANTOS41,ZILDAMACHADOPAZZETTODOSSANTOS43,ZILDAMACHADOPAZZETTODOSSANTOS27052021
1084379,1176620,-1,PR,411920,410970,ZULEIDE MENDES DE ALMEIDA,F,55,"COVID-19, Biologia Molecular",2021-05-26,2021-05-28,2021-05-21 00:00:00,3.0,NaT,NaT,ZULEIDEMENDESDEALMEIDA55,ZULEIDEMENDESDEALMEIDA55,ZULEIDEMENDESDEALMEIDA54,ZULEIDEMENDESDEALMEIDA56,ZULEIDEMENDESDEALMEIDA26052021


In [6]:
novos_casos = notifica.loc[~(
    (notifica['hash'].isin(casosct['hash'])) |
    (notifica['hash'].isin(casosct['hash_less'])) |
    (notifica['hash'].isin(casosct['hash_more'])) |
    (notifica['hash_diag'].isin(casosct['hash_diag']))
)].copy()

novos_casos['data_com'] = pd.to_datetime(date.today()).strftime('%d/%m/%Y')
novos_casos = novos_casos.sort_values('paciente')
novos_casos

,id,origem,data_notificacao,updated_at,paciente,sexo,nome_mae,idade,data_nascimento,pais_residencia,uf_residencia,ibge_residencia,uf_unidade_notifica,ibge_unidade_notifica,exame,criterio_classificacao,data_liberacao,data_coleta,data_recebimento,data_1o_sintomas,evolucao,data_cura_obito,hash,hash_atend,hash_less,hash_more,hash_diag,rs,mun_resid,uf,municipio,mun_atend,data_com
70212,3946446,1,30/03/2021,29/05/2021,ABEGAIL CASTANHO COELHO,F,ANTONIA PEDROSO DA SILVA,86,28/07/1934,1,41,411820,41,411820,Vírus Respiratório,5,2021-04-01,30/03/2021,31/03/2021,24/03/2021,1,2021-04-04,ABEGAILCASTANHOCOELHO86411820,ABEGAILCASTANHOCOELHO86411820,ABEGAILCASTANHOCOELHO85411820,ABEGAILCASTANHOCOELHO87411820,ABEGAILCASTANHOCOELHO01042021,1,PARANAGUA,PR,Paranaguá,PARANAGUA,29/05/2021
28429,3753806,1,19/05/2021,28/05/2021,ABEL DE SOUZA,M,ROSI APARECIDA DE SOUZA,42,27/11/1978,1,41,410580,41,999999,"COVID-19, Biologia Molecular",1,2021-05-22,16/05/2021,22/05/2021,18/05/2021,3,NaT,ABELDESOUZA42410580,ABELDESOUZA42999999,ABELDESOUZA41410580,ABELDESOUZA43410580,ABELDESOUZA22052021,2,COLOMBO,PR,Colombo,IGNORADO/EXTERIOR/IG,29/05/2021
50051,3854525,1,25/05/2021,28/05/2021,ABEL LANKOVSKI,M,TECLA PORANIAK,71,07/01/1950,1,41,410170,41,999999,"COVID-19, Biologia Molecular",1,2021-05-28,25/05/2021,27/05/2021,22/05/2021,3,NaT,ABELLANKOVSKI71410170,ABELLANKOVSKI71999999,ABELLANKOVSKI70410170,ABELLANKOVSKI72410170,ABELLANKOVSKI28052021,11,ARARUNA,PR,Araruna,IGNORADO/EXTERIOR/IG,29/05/2021
25892,3733934,1,18/05/2021,28/05/2021,ABNER GILMAR DA SILVA SANTOS,M,NILCEIA APARECIDA DA SILVA,20,02/08/2000,1,41,412240,41,412240,"COVID-19, Biologia Molecular",1,2021-05-27,21/05/2021,27/05/2021,17/05/2021,3,NaT,ABNERGILMARDASILVASANTOS20412240,ABNERGILMARDASILVASANTOS20412240,ABNERGILMARDASILVASANTOS19412240,ABNERGILMARDASILVASANTOS21412240,ABNERGILMARDASILVASANTOS27052021,17,ROLANDIA,PR,Rolândia,ROLANDIA,29/05/2021
67549,3933194,1,28/05/2021,28/05/2021,ABRAAO DE AZEVEDO,M,MARIA MALVINA FERREIRA DE AZEVEDO,45,08/08/1975,1,41,412770,41,412770,Teste Rápido Antígeno (Imunocromatografia),1,2021-05-28,28/05/2021,NaN,25/05/2021,3,NaT,ABRAAODEAZEVEDO45412770,ABRAAODEAZEVEDO45412770,ABRAAODEAZEVEDO44412770,ABRAAODEAZEVEDO46412770,ABRAAODEAZEVEDO28052021,20,TOLEDO,PR,Toledo,TOLEDO,29/05/2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11620,3573420,1,08/05/2021,26/05/2021,ZILMA APARECIDA DA SIVA CABREIRA,F,MARIA APARECIDA DA SILVA,49,09/10/1971,1,41,410655,41,999999,Teste Rápido Antígeno (Imunocromatografia),1,2021-05-10,10/05/2021,NaN,07/05/2021,1,2021-05-16,ZILMAAPARECIDADASIVACABREIRA49410655,ZILMAAPARECIDADASIVACABREIRA49999999,ZILMAAPARECIDADASIVACABREIRA48410655,ZILMAAPARECIDADASIVACABREIRA50410655,ZILMAAPARECIDADASIVACABREIRA10052021,11,CORUMBATAI DO SUL,PR,Corumbataí do Sul,IGNORADO/EXTERIOR/IG,29/05/2021
65594,3923416,1,28/05/2021,29/05/2021,ZILTON ANTONIO LEITE,M,SEBASTIANA LEITE,31,04/07/1989,1,41,410850,41,410850,Teste Rápido Antígeno (Imunocromatografia),1,2021-05-29,29/05/2021,29/05/2021,23/05/2021,3,NaT,ZILTONANTONIOLEITE31410850,ZILTONANTONIOLEITE31410850,ZILTONANTONIOLEITE30410850,ZILTONANTONIOLEITE32410850,ZILTONANTONIOLEITE29052021,6,GENERAL CARNEIRO,PR,General Carneiro,GENERAL CARNEIRO,29/05/2021
51919,3861326,1,25/05/2021,28/05/2021,ZORAIDE ESTEFANO DA SILVA,F,LEONILDA ELEUTERIA DOS SANTOS,71,10/12/1949,1,41,411915,41,999999,"COVID-19, Biologia Molecular",1,2021-05-27,25/05/2021,26/05/2021,23/05/2021,3,NaT,ZORAIDEESTEFANODASILVA71411915,ZORAIDEESTEFANODASILVA71999999,ZORAIDEESTEFANODASILVA70411915,ZORAIDEESTEFANODASILVA72411915,ZORAIDEESTEFANODASILVA27052021,2,PINHAIS,PR,Pinhais,IGNORADO/EXTERIOR/IG,29/05/2021
61155,3900570,10,27/05/2021,28/05/2021,ZULEIDE PEREIRA DE CARVALHO,F,MARIA GLEIDE DE CARVALHO,47,14/08/1973,1,41,411520,41,411520,"COVID-19, Biologia Molecular",1,2021-05-28,27/05/2021,NaN,20/05/2021,3,NaT,ZULEIDEPEREIRADECARVALHO47411520,ZULEIDEPEREIRADECARVALHO4

In [7]:
novos_casos_errados = novos_casos.loc[(
    (novos_casos['data_liberacao'] <= pd.to_datetime('2020-03-01')) |
    (novos_casos['data_liberacao'] > pd.to_datetime(date.today()))
)]

novos_casos = novos_casos.loc[~(
    (novos_casos['data_liberacao'] <= pd.to_datetime('2020-03-01')) |
    (novos_casos['data_liberacao'] > pd.to_datetime(date.today()))
)]

novos_casos[['id','paciente','sexo','idade','mun_resid', 'mun_atend', 'rs', 'exame','data_liberacao','data_com','data_1o_sintomas','hash']].to_excel(join(default_output,'novos_casos.xlsx'))

novos_casos_errados

,id,origem,data_notificacao,updated_at,paciente,sexo,nome_mae,idade,data_nascimento,pais_residencia,uf_residencia,ibge_residencia,uf_unidade_notifica,ibge_unidade_notifica,exame,criterio_classificacao,data_liberacao,data_coleta,data_recebimento,data_1o_sintomas,evolucao,data_cura_obito,hash,hash_atend,hash_less,hash_more,hash_diag,rs,mun_resid,uf,municipio,mun_atend,data_com


In [8]:
obitos_notifica = notifica.loc[notifica['evolucao']==2]
obitos_casos = casosct.loc[casosct['evolucao']==2]

novos_obitos = obitos_notifica.loc[~(
    (obitos_notifica['hash'].isin(obitos_casos['hash'])) |
    (obitos_notifica['hash'].isin(obitos_casos['hash_less'])) |
    (obitos_notifica['hash'].isin(obitos_casos['hash_more'])) |
    (obitos_notifica['hash_diag'].isin(obitos_casos['hash_diag'])) 
)]

novos_obitos = novos_obitos.loc[novos_obitos['data_cura_obito'].notna()]
novos_obitos = novos_obitos.loc[novos_obitos['hash'].isin(casosct['hash'].tolist() + novos_casos['hash'].tolist())]
novos_obitos = novos_obitos.sort_values('paciente')
novos_obitos

,id,origem,data_notificacao,updated_at,paciente,sexo,nome_mae,idade,data_nascimento,pais_residencia,uf_residencia,ibge_residencia,uf_unidade_notifica,ibge_unidade_notifica,exame,criterio_classificacao,data_liberacao,data_coleta,data_recebimento,data_1o_sintomas,evolucao,data_cura_obito,hash,hash_atend,hash_less,hash_more,hash_diag,rs,mun_resid,uf,municipio,mun_atend
17674,3655337,1,12/05/2021,26/05/2021,ADAIR CAPOANI,M,CLARICE CAPOANI,62,21/05/1959,1,41,410840,41,410840,Vírus Respiratório,1,2021-05-21,19/05/2021,21/05/2021,10/05/2021,2,2021-05-24,ADAIRCAPOANI62410840,ADAIRCAPOANI62410840,ADAIRCAPOANI61410840,ADAIRCAPOANI63410840,ADAIRCAPOANI21052021,8,FRANCISCO BELTRAO,PR,Francisco Beltrão,FRANCISCO BELTRAO
25758,3732851,10,06/04/2021,26/05/2021,ADELAIDE BENITE,F,ZORAIDE BENITE,67,29/11/1953,1,41,411750,41,411520,"COVID-19, Biologia Molecular",1,2021-04-09,06/04/2021,NaN,02/04/2021,2,2021-04-12,ADELAIDEBENITE67411750,ADELAIDEBENITE67411520,ADELAIDEBENITE66411750,ADELAIDEBENITE68411750,ADELAIDEBENITE09042021,15,PAICANDU,PR,Paiçandu,MARINGA
11732,3575967,1,08/05/2021,24/05/2021,ALAIRTO BUENO,M,MARIA APARECIDA BUENO,52,22/11/1968,1,41,412770,41,412770,Vírus Respiratório,1,2021-05-10,08/05/2021,10/05/2021,24/04/2021,2,2021-05-23,ALAIRTOBUENO52412770,ALAIRTOBUENO52412770,ALAIRTOBUENO51412770,ALAIRTOBUENO53412770,ALAIRTOBUENO10052021,20,TOLEDO,PR,Toledo,TOLEDO
3376,2763042,1,09/03/2021,25/05/2021,ALCIDES FRANCISCO DA SILVA,M,NAIR DOS SANTOS DA SILVA,67,17/07/1953,1,41,411420,41,411420,Teste Rápido Antígeno (Imunocromatografia),1,2021-03-06,06/03/2021,06/03/2021,06/03/2021,2,2021-04-01,ALCIDESFRANCISCODASILVA67411420,ALCIDESFRANCISCODASILVA67411420,ALCIDESFRANCISCODASILVA66411420,ALCIDESFRANCISCODASILVA68411420,ALCIDESFRANCISCODASILVA06032021,15,MANDAGUARI,PR,Mandaguari,MANDAGUARI
2812,2469345,1,21/02/2021,29/05/2021,ALCIDES JOSE LOVATTO,M,HERONDINA LEONOR LOVATTO,72,14/04/1949,1,41,411850,41,411850,"COVID-19, Biologia Molecular",1,2021-02-17,17/02/2021,17/02/2021,10/02/2021,2,2021-03-08,ALCIDESJOSELOVATTO72411850,ALCIDESJOSELOVATTO72411850,ALCIDESJOSELOVATTO71411850,ALCIDESJOSELOVATTO73411850,ALCIDESJOSELOVATTO17022021,7,PATO BRANCO,PR,Pato Branco,PATO BRANCO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9587,3490024,10,03/05/2021,27/05/2021,WILSON TRAMONTINI,M,MARIA DO CARMO TRAMONTINI,82,14/02/1939,1,41,410550,41,411520,"COVID-19, Biologia Molecular",1,2021-05-02,02/05/2021,02/05/2021,26/04/2021,2,2021-05-24,WILSONTRAMONTINI82410550,WILSONTRAMONTINI82411520,WILSONTRAMONTINI81410550,WILSONTRAMONTINI83410550,WILSONTRAMONTINI02052021,13,CIANORTE,PR,Cianorte,MARINGA
29159,3760586,1,20/05/2021,25/05/2021,ZAIDI PEPE DE OLIVEIRA,F,MARIA AUGUSTA GONCALVES,65,05/08/1955,1,41,411070,41,411070,"COVID-19, Biologia Molecular",1,2021-05-21,19/05/2021,21/05/2021,15/05/2021,2,2021-05-25,ZAIDIPEPEDEOLIVEIRA65411070,ZAIDIPEPEDEOLIVEIRA65411070,ZAIDIPEPEDEOLIVEIRA64411070,ZAIDIPEPEDEOLIVEIRA66411070,ZAIDIPEPEDEOLIVEIRA21052021,4,IRATI,PR,Irati,IRATI
14104,3612684,1,11/05/2021,27/05/2021,ZENILDA IENSEN EMILIANO,F,IOLANDA LOPES IENSEN,42,04/05/1979,1,41,410895,41,410895,"COVID-19, Biologia Molecular",1,2021-05-13,11/05/2021,12/05/2021,08/05/2021,2,2021-05-27,ZENILDAIENSENEMILIANO42410895,ZENILDAIENSENEMILIANO42410895,ZENILDAIENSENEMILIANO41410895,ZENILDAIENSENEMILIANO43410895,ZENILDAIENSENEMILIANO13052021,4,GUAMIRANGA,PR,Guamiranga,GUAMIRANGA
5019,3060727,10,26/03/2021,25/05/2021,ZILDA DOS SANTOS VICENTE,F,OLIVIA BALDAVIA DOS SANTOS,72,06/05/1949,1,41,411520,41,411520,Vírus Respiratório,1,2021-04-02,31/03/2021,02/04/2021,22/03/2021,2,2021-04-05,ZILDADOSSANTOSVICENTE72411520,ZILDADOSSANTOSVICENTE72411520,ZILDADOSSANTOSVICENTE71411520,ZILDADOSSANTOSVICENTE73411520,ZILDADOSSANTOSVICENTE02042021,15,MARINGA,PR,Maringá,MARINGA


In [9]:
novos_obitos_errados = novos_obitos.loc[(
    (novos_obitos['data_cura_obito'] <= pd.to_datetime('2020-03-01')) |
    (novos_obitos['data_cura_obito'] > pd.to_datetime(date.today()))
)]

novos_obitos = novos_obitos.loc[~(
    (novos_obitos['data_cura_obito'] <= pd.to_datetime('2020-03-01')) |
    (novos_obitos['data_cura_obito'] > pd.to_datetime(date.today()))
)]

novos_obitos[['id','paciente','sexo','idade','mun_resid', 'rs', 'data_cura_obito','hash']].to_excel(join(default_output,'novos_obitos.xlsx'))

novos_obitos_errados

,id,origem,data_notificacao,updated_at,paciente,sexo,nome_mae,idade,data_nascimento,pais_residencia,uf_residencia,ibge_residencia,uf_unidade_notifica,ibge_unidade_notifica,exame,criterio_classificacao,data_liberacao,data_coleta,data_recebimento,data_1o_sintomas,evolucao,data_cura_obito,hash,hash_atend,hash_less,hash_more,hash_diag,rs,mun_resid,uf,municipio,mun_atend
10548,3535459,1,05/05/2021,28/05/2021,PEDRO RIBEIRO MARTINS,M,EMILIA RIBEIRO MARTINS,70,12/08/1950,1,41,411010,41,411990,Vírus Respiratório,1,2021-05-10,06/05/2021,07/05/2021,19/04/2021,2,2021-08-28,PEDRORIBEIROMARTINS70411010,PEDRORIBEIROMARTINS70411990,PEDRORIBEIROMARTINS69411010,PEDRORIBEIROMARTINS71411010,PEDRORIBEIROMARTINS10052021,4,IMBITUVA,PR,Imbituva,PONTA GROSSA


In [10]:
casos_confirmados = pd.merge(casosct.rename(columns={'ibge_resid':'ibge'}),municipios[['ibge','rs']],how='left',on='ibge')

casos_confirmadosPR = casos_confirmados.loc[casos_confirmados['rs']!=99]

obitos_confirmados =  casos_confirmados.loc[casos_confirmados['evolucao']==2]

obitos_confirmadosPR = obitos_confirmados.loc[obitos_confirmados['rs']!=99]

print(f"Total de casos: {len(casos_confirmados)} + {len(novos_casos)}")
print(f"Total de obitos: {len(obitos_confirmados)} + {len(novos_obitos)}\n\n")

novos_casosPR = novos_casos.loc[novos_casos['rs']!=99].copy()
print(f"Total de casos PR: {len(casos_confirmadosPR)} + {len(novos_casosPR)}")

novos_obitosPR = novos_obitos.loc[novos_obitos['rs']!=99].copy()
print(f"Total de obitos PR: {len(obitos_confirmadosPR)} + {len(novos_obitosPR)}")

novos_casosFora = novos_casos.loc[novos_casos['rs']==99].copy()
print(f"Total de casos Fora: {len(casos_confirmados) - len(casos_confirmadosPR)} + {len(novos_casosFora)}")

novos_obitosFora = novos_obitos.loc[novos_obitos['rs']==99].copy()
print(f"Total de obitos Fora: {len(obitos_confirmados) - len(obitos_confirmadosPR)} + {len(novos_obitosFora)}")


novos_obitosPR_group = novos_obitosPR.groupby(by='municipio')

today = datetime.today()
ontem = today - timedelta(1)
anteontem = ontem - timedelta(1)
data_retroativos = ontem - timedelta(14)

retroativos = novos_casosPR.loc[(novos_casosPR['data_liberacao'].apply(date_parser) <= data_retroativos)].sort_values(by='data_liberacao')
last2weeks = novos_casosPR.loc[(novos_casosPR['data_liberacao'].apply(date_parser) > data_retroativos) & (novos_casosPR['data_liberacao'].apply(date_parser) <= anteontem)].sort_values(by='data_liberacao')
hoje = novos_casosPR.loc[(novos_casosPR['data_liberacao'].apply(date_parser) > anteontem)].sort_values(by='data_liberacao')

obitos_retroativos = novos_obitosPR.loc[(novos_obitosPR['data_cura_obito'].apply(date_parser) <= data_retroativos)].sort_values(by='data_cura_obito')
obitos_last2weeks = novos_obitosPR.loc[(novos_obitosPR['data_cura_obito'].apply(date_parser) > data_retroativos) & (novos_obitosPR['data_cura_obito'].apply(date_parser) <= anteontem)].sort_values(by='data_cura_obito')
obitos_hoje = novos_obitosPR.loc[(novos_obitosPR['data_cura_obito'].apply(date_parser) > anteontem)].sort_values(by='data_cura_obito')

relatorio_file = join(default_output,f"relatorio_{(today.strftime('%d/%m/%Y_%Hh').replace('/','_').replace(' ',''))}.txt")
with codecs.open(relatorio_file,"w","utf-8-sig") as relatorio:
    relatorio.write(f"{today.strftime('%d/%m/%Y')}\n")
    relatorio.write(f"{len(novos_casosPR):,} novos casos residentes ".replace(',','.'))

    if len(novos_casosFora) > 0:
        relatorio.write(f"e {len(novos_casosFora):,} não residente{'s' if len(novos_casosFora) > 1 else ''} ".replace(',','.'))
    relatorio.write(f"divulgados no PR.\n")

    relatorio.write(f"{len(casos_confirmadosPR)+len(novos_casosPR):,} casos confirmados residentes do PR.\n".replace(',','.'))
    relatorio.write(f"{len(casos_confirmados)+len(novos_casos):,} total geral.\n\n".replace(',','.'))
    relatorio.write(f"{len(novos_obitosPR):,} Óbitos residentes do PR:\n".replace(',','.'))

    for municipio, obitos in novos_obitosPR_group:
        relatorio.write(f"{len(obitos):,} {municipio}\n".replace(',','.'))

    if len(novos_obitosFora) > 0:
        relatorio.write('\n')
        relatorio.write(f"{len(novos_obitosFora):,} Óbito{'s' if len(novos_obitosFora) > 1 else ''} não residente{'s' if len(novos_obitosFora) > 1 else ''} do PR.\n".replace(',','.'))

    relatorio.write('\n')
    relatorio.write(f"{len(obitos_confirmadosPR)+len(novos_obitosPR):,} óbitos residentes do PR.\n".replace(',','.'))
    relatorio.write(f"{len(obitos_confirmados)+len(novos_obitos):,} total geral.\n\n".replace(',','.'))

    for _, row in novos_obitos.iterrows():
       relatorio.write(f"{row['sexo']}\t{row['idade']}\t{row['municipio'] if row['rs']!=99 else (row['municipio']+'/'+row['uf'])}\t{str(row['rs']).zfill(2) if row['rs']!=99 else '#N/D'}\t{row['data_cura_obito'].day}/{static.meses[row['data_cura_obito'].month-1]}/{row['data_cura_obito'].year}\n")
    relatorio.write('\n')

    if True:

        #casos
        relatorio.write(f"{len(novos_casosPR):,} novos casos residentes divulgados no PR.\n".replace(',','.'))

        relatorio.write(f"{len(retroativos):,} casos retroativos confirmados no período de {retroativos.iloc[0]['data_liberacao'].strftime('%d/%m/%Y')} à {retroativos.iloc[-1]['data_liberacao'].strftime('%d/%m/%Y')}.\n".replace(',','.'))
        relatorio.write(f"{len(last2weeks):,} novos casos confirmados no período de {last2weeks.iloc[0]['data_liberacao'].strftime('%d/%m/%Y')} à {last2weeks.iloc[-1]['data_liberacao'].strftime('%d/%m/%Y')}.\n".replace(',','.'))
        relatorio.write(f"{len(hoje):,} novos casos confirmados hoje.\n\n".replace(',','.'))


        novos_casosPR['month'] = novos_casosPR.apply(lambda x: x['data_liberacao'].month, axis=1)
        novos_casosPR['year'] = novos_casosPR.apply(lambda x: x['data_liberacao'].year, axis=1)
        relatorio.write('Novos casos por meses:\n')
        for group, value in novos_casosPR.groupby(by=['year','month']):
            relatorio.write(f"{static.meses[int(group[1])-1]}\\{group[0]}: {len(value)}\n")
        relatorio.write('\n')

        #obitos
        relatorio.write(f"{len(novos_obitosPR):,} novos obitos residentes divulgados no PR.\n".replace(',','.'))

        relatorio.write(f"{len(obitos_retroativos):,} obitos retroativos ocorridos no período de {obitos_retroativos.iloc[0]['data_cura_obito'].strftime('%d/%m/%Y')} à {obitos_retroativos.iloc[-1]['data_cura_obito'].strftime('%d/%m/%Y')}.\n".replace(',','.'))
        relatorio.write(f"{len(obitos_last2weeks):,} novos obitos ocorridos no período de {obitos_last2weeks.iloc[0]['data_cura_obito'].strftime('%d/%m/%Y')} à {obitos_last2weeks.iloc[-1]['data_cura_obito'].strftime('%d/%m/%Y')}.\n".replace(',','.'))
        relatorio.write(f"{len(obitos_hoje):,} novos obitos ocorridos hoje.\n\n".replace(',','.'))

        relatorio.write('Novos obitos por meses:\n')
        novos_obitosPR['month'] = novos_obitosPR.apply(lambda x: x['data_cura_obito'].month, axis=1)
        novos_obitosPR['year'] = novos_obitosPR.apply(lambda x: x['data_cura_obito'].year, axis=1)
        for group, value in novos_obitosPR.groupby(by=['year','month']):
            relatorio.write(f"{static.meses[int(group[1])-1]}\\{group[0]}: {len(value)}\n")
        relatorio.write('\n')

        relatorio.write('Novos obitos por dia:\n')
        for group, value in novos_obitosPR.groupby(by='data_cura_obito'):
            relatorio.write(f"{group.strftime('%d/%m/%Y')}: {len(value)}\n")

        #-----RELATÓRIO DA COMUNICAÇÃO--------------
        obitos_list = []
        munic = []
        for municipio, obitos in novos_obitosPR_group:
            obito = len(obitos)
            obitos_list.append(obito)
            munic.append(municipio)             

        dicionario = (dict(zip(list(munic),list(obitos_list))))
        #print(dicionario)
        dicionario = sorted(dicionario.items(),key=lambda x: x[1], reverse = True)
        #print(dicionario)


        relatorio.write(f"\nOs pacientes que foram a óbito residiam em: ")
        for municip, obit in dict(dicionario).items():
            if obit != 1:
                relatorio.write(f"{municip} ({obit})")
                relatorio.write(f", ")
        relatorio.write(f".\n")
        relatorio.write(f"A Sesa registra ainda a morte de uma pessoa que residia em cada um dos seguintes municípios:")
        for municip, obit in dict(dicionario).items():
            if obit == 1:
                relatorio.write(f"{municip}")
                relatorio.write(f", ")


Total de casos: 1084380 + 7749
Total de obitos: 26168 + 258


Total de casos PR: 1078504 + 7735
Total de obitos PR: 26021 + 256
Total de casos Fora: 5876 + 14
Total de obitos Fora: 147 + 2


In [11]:
with codecs.open(relatorio_file,"r","utf-8-sig") as relatorio:
    print(relatorio.read())

29/05/2021
7.735 novos casos residentes e 14 não residentes divulgados no PR.
1.086.239 casos confirmados residentes do PR.
1.092.129 total geral.

256 Óbitos residentes do PR:
1 Almirante Tamandaré
11 Apucarana
3 Arapongas
1 Araucária
1 Astorga
1 Bandeirantes
1 Barracão
1 Bela Vista do Paraíso
2 Boa Vista da Aparecida
1 Califórnia
2 Campo Largo
2 Campo Mourão
1 Campo do Tenente
1 Capitão Leônidas Marques
2 Carambeí
4 Cascavel
5 Castro
2 Cianorte
3 Colombo
1 Congonhinhas
1 Cornélio Procópio
9 Curitiba
4 Dois Vizinhos
1 Engenheiro Beltrão
1 Enéas Marques
2 Fazenda Rio Grande
1 Floraí
1 Formosa do Oeste
7 Foz do Iguaçu
16 Francisco Beltrão
1 Guamiranga
8 Guarapuava
2 Guaratuba
2 Guaíra
1 Ibaiti
1 Ibiporã
1 Imbaú
2 Imbituva
1 Ipiranga
9 Irati
1 Itapejara d'Oeste
1 Ivaí
3 Jaguariaíva
1 Jataizinho
15 Londrina
2 Mandaguari
1 Mandirituba
4 Marialva
1 Marilena
1 Mariluz
15 Maringá
1 Matinhos
1 Medianeira
2 Moreira Sales
1 Nova Esperança
1 Ortigueira
3 Paiçandu
1 Palmas
2 Palmeira
1 Palmital
5 

In [ ]:
while input('bateu?') != 'bateu': pass

In [ ]:
novos_casos = novos_casos.rename(columns={
    'id':'id_notifica',
    'ibge_residencia': 'ibge_resid',
    'ibge_unidade_notifica': 'ibge_atend',
    'paciente':'nome',
    'sexo': 'sexo',
    'idade':'idade',
    'exame': 'laboratorio',
    'data_liberacao':'dt_diag',
    'data_com':'comunicacao',
    'data_1o_sintomas': 'is',
    'evolucao': 'evolucao',
    'data_cura_obito': 'data_evolucao',
    'uf': 'uf_resid'
})

novos_casos = novos_casos[['id_notifica', 'uf_resid', 'ibge_resid', 'ibge_atend', 'nome', 'sexo', 'idade', 'laboratorio', 'dt_diag', 'comunicacao', 'is', 'evolucao', 'data_evolucao']]

novos_casos.loc[novos_casos['evolucao']==5,'evolucao'] = 2
novos_casos.loc[~novos_casos['evolucao'].isin([1,2,3]),'evolucao'] = 3

novos_casos['hash'] = (novos_casos['nome'].apply(normalize_hash) +
                  novos_casos['idade'].astype(str) +
                  novos_casos['ibge_resid'].astype(str) )

novos_casos['hash_less'] = ( novos_casos['nome'].apply(normalize_hash) +
                        novos_casos['idade'].apply(lambda x: str(x - 1)) +
                        novos_casos['ibge_resid'].astype(str) )

novos_casos['hash_more'] = ( novos_casos['nome'].apply(normalize_hash) +
                        novos_casos['idade'].apply(lambda x: str(x + 1)) +
                        novos_casos['ibge_resid'].astype(str) )

novos_casos['identificacao'] = list(range(casosct.iloc[-1,0] + 1,casosct.iloc[-1,0] + 1 + len(novos_casos)))
casosct = casosct.append(novos_casos)
print(len(casosct))
casosct.groupby('evolucao')[['sexo']].count()

In [ ]:
novos_obitos = novos_obitos.rename(columns={
    'id': 'id_notifica',
    'data_cura_obito': 'data_evolucao',
    'paciente':'nome'
})

novos_obitos['hash'] = (novos_obitos['nome'].apply(normalize_hash) +
                          novos_obitos['idade'].astype(str) +
                              novos_obitos['ibge_residencia'].astype(str) )

novos_obitos['data_com_evolucao'] = pd.to_datetime(date.today())
novos_obitos['evolucao'] = 2

novos_obitos = novos_obitos[['hash','id_notifica','evolucao','data_evolucao','data_com_evolucao']]
novos_obitos = novos_obitos.set_index('hash')
casosct = casosct.set_index('hash')
casosct.update(novos_obitos)
casosct.reset_index(inplace=True)
casosct = casosct[['identificacao','id_notifica','uf_resid','ibge_resid','ibge_atend','nome','sexo','idade','laboratorio','dt_diag','comunicacao','is','evolucao','data_evolucao','data_com_evolucao','hash','hash_less','hash_more','hash_diag']]

print(len(casosct))
casosct.groupby('evolucao')[['sexo']].count()

In [ ]:
cc.df = casosct
cc.save
casosct.tail()

In [ ]:
municipios = Municipios()

macros_idx_labels = dict([ [i,x] for (i,x) in enumerate(['Leste','Oeste','Noroeste','Norte','Fora'])])
macros_labels_idx = dict([ [x,i] for (i,x) in macros_idx_labels.items() ])
municipios['macro'] = municipios['macro'].apply(lambda x: macros_labels_idx[x])

casosct = pd.merge(casosct.rename(columns={'ibge_resid':'ibge'}),municipios,on=['ibge'],how='left').rename(columns={'ibge':'ibge_resid'})

casosct['municipio_pr'] = casosct['municipio']
casosct.loc[casosct['uf_resid']!='PR','municipio_pr'] = 'Fora'

casosct.loc[casosct['uf_resid']!='PR','ibge7'] = 9999999

casosct['uf_pr'] = 'PR'
casosct.loc[casosct['uf_resid']!='PR','uf_pr'] = 'Fora'

In [ ]:
pd.options.display.float_format = lambda x: '{:,.0f}'.format(x).replace(',','.')

total = casosct.groupby(['evolucao','uf_pr'])[['sexo']].count().unstack().droplevel(0,1)[['PR','Fora']]
total['GERAL'] = total.sum(1)
total = total.append(pd.Series(total.sum(0),name=0))
total.index = [2,1,3,0]
total = total.sort_index()
total = total.rename(index={0:'CASOS',2:'RECUPERADOS',1:'OBITOS',3:'ATIVOS'})
total.index.name = None
total.columns.name = None

total.astype(float)

In [ ]:
total_macros = casosct.groupby(['macro','evolucao'])[['sexo']].count().unstack()
total_macros.columns = total_macros.columns.droplevel(0)

total_macros['CASOS'] = total_macros.sum(1)
total_macros = total_macros.fillna(0)
total_macros = total_macros.rename(columns={1:'RECUPERADOS',2:'OBITOS',3:'ATIVOS'})
total_macros = total_macros[['CASOS','OBITOS','RECUPERADOS','ATIVOS']]

total_macros.index = [ macros_idx_labels[x] if isinstance(x,int) else x for x in total_macros.index]

total_macros.astype(float)

In [ ]:
total_regionais = casosct.groupby(['rs','regional','evolucao'])[['sexo']].count().unstack()
total_regionais.columns = total_regionais.columns.droplevel(0)

total_regionais['CASOS'] = total_regionais.sum(1)
total_regionais = total_regionais.fillna(0)
total_regionais = total_regionais.rename(columns={1:'RECUPERADOS',2:'OBITOS',3:'ATIVOS'})
total_regionais = total_regionais[['CASOS','OBITOS','RECUPERADOS','ATIVOS']]

total_regionais.astype(float)

In [ ]:
total_municipios = casosct.groupby(['macro','rs','regional','ibge7','municipio_pr','evolucao'])[['sexo']].count().unstack()

total_municipios[('','CASOS')] = total_municipios.sum(1)
total_municipios = total_municipios.fillna(0).astype(float)
total_municipios.columns = total_municipios.columns.droplevel(0)
total_municipios = total_municipios.rename(columns={1:'RECUPERADOS',2:'OBITOS',3:'ATIVOS'})
total_municipios = total_municipios[['CASOS','OBITOS','RECUPERADOS','ATIVOS']]

total_municipios.index = total_municipios.index.set_levels([ macros_idx_labels[x] if isinstance(x,int) else x for x in total_municipios.index.levels[0]], level=0)

total_municipios

In [ ]:
ministerio_cols = ['rs','municipio_pr','CASOS','OBITOS']
total_municipios.reset_index()[ministerio_cols].sort_values(['rs','municipio_pr']).to_excel(join(default_output,'ministerio.xlsx'),index=False)

In [ ]:
casosct['regional'] = casosct['rs'].apply(lambda x: str(x).zfill(2)) +'ª '+ casosct['regional']
casos_confirmados = casosct[['id_notifica','macro','regional','ibge_resid','municipio','ibge_atend','nome','sexo','idade', 'laboratorio', 'dt_diag', 'comunicacao', 'is', 'evolucao','data_evolucao', 'data_com_evolucao']].copy()
casos_confirmados.loc[casos_confirmados['evolucao']==1,'evolucao'] = 'CURA'
casos_confirmados.loc[casos_confirmados['evolucao']==2,'evolucao'] = 'OBITO'
casos_confirmados.loc[casos_confirmados['evolucao']==3,'evolucao'] = 'ATIVO'

In [ ]:
obitos_confirmados = casos_confirmados.loc[casos_confirmados['evolucao']=='OBITO']

In [ ]:
writer = pd.ExcelWriter(join(default_output, "casos_confirmados.xlsx"),
                    engine='xlsxwriter',
                    datetime_format='dd/mm/yyyy',
                    date_format='dd/mm/yyyy')

total.to_excel(writer,'total')
total_macros.to_excel(writer,'total_macros')
total_regionais.to_excel(writer,'total_regionais')
total_municipios.to_excel(writer,'total_municipios')

fit_cols(writer,total,'total')
fit_cols(writer,total_macros,'total_macros')
fit_cols(writer,total_regionais,'total_regionais')
fit_cols(writer,total_municipios,'total_municipios')

for idx, macro in macros_idx_labels.items():
    casos_macro = casos_confirmados.loc[casos_confirmados['macro']==idx].copy()
    casos_macro['macro'] = macro
    casos_macro.to_excel(writer,macro)
    fit_cols(writer,casos_macro,macro)


writer.save()

In [ ]:
writer = pd.ExcelWriter(join(default_output, "obitos_confirmados.xlsx"),
                    engine='xlsxwriter',
                    datetime_format='dd/mm/yyyy',
                    date_format='dd/mm/yyyy')

total.to_excel(writer,'total')
total_macros.to_excel(writer,'total_macros')
total_regionais.to_excel(writer,'total_regionais')
total_municipios.to_excel(writer,'total_municipios')

fit_cols(writer,total,'total')
fit_cols(writer,total_macros,'total_macros')
fit_cols(writer,total_regionais,'total_regionais')
fit_cols(writer,total_municipios,'total_municipios')

for idx, macro in macros_idx_labels.items():
    casos_macro = obitos_confirmados.loc[obitos_confirmados['macro']==idx].copy()
    casos_macro['macro'] = macro
    casos_macro.to_excel(writer,macro)
    fit_cols(writer,casos_macro,macro)


writer.save()

In [ ]:
writer = pd.ExcelWriter(join(default_output,"casos_confirmados_regionais.xlsx"),
                    engine='xlsxwriter',
                    datetime_format='dd/mm/yyyy',
                    date_format='dd/mm/yyyy')

total.to_excel(writer,'total')
total_macros.to_excel(writer,'total_macros')
total_regionais.to_excel(writer,'total_regionais')
total_municipios.to_excel(writer,'total_municipios')

fit_cols(writer,total,'total')
fit_cols(writer,total_macros,'total_macros')
fit_cols(writer,total_regionais,'total_regionais')
fit_cols(writer,total_municipios,'total_municipios')

for regional in sorted(casos_confirmados['regional'].unique()):
    casos_regional = casos_confirmados.loc[casos_confirmados['regional']==regional].copy()
    casos_regional['macro'] = casos_regional.apply(lambda x: macros_idx_labels[x])
    casos_regional.to_excel(writer,regional)
    fit_cols(writer,casos_regional,regional)


writer.save()

In [ ]:
writer = pd.ExcelWriter(join(default_output,"obitos_confirmados_regionais.xlsx"),
                    engine='xlsxwriter',
                    datetime_format='dd/mm/yyyy',
                    date_format='dd/mm/yyyy')

total.to_excel(writer,'total')
total_macros.to_excel(writer,'total_macros')
total_regionais.to_excel(writer,'total_regionais')
total_municipios.to_excel(writer,'total_municipios')

fit_cols(writer,total,'total')
fit_cols(writer,total_macros,'total_macros')
fit_cols(writer,total_regionais,'total_regionais')
fit_cols(writer,total_municipios,'total_municipios')

for regional in sorted(obitos_confirmados['regional'].unique()):
    obitos_regional = obitos_confirmados.loc[obitos_confirmados['regional']==regional].copy()
    obitos_regional['macro'] = obitos_regional.apply(lambda x: macros_idx_labels[x])
    obitos_regional.to_excel(writer,regional)
    fit_cols(writer,obitos_regional,regional)


writer.save()

In [ ]:
cc = CasosConfirmados()
cc.load()

In [ ]:
casos = cc.df
casos['is'] = casos['is'].apply(lambda x: str(x)[:10] if not str(x).lower() in ['none','nat','nan'] else 'none')
len(casos.loc[casos['is']!='none'])

In [ ]:
casos.loc[casos['is'].str.contains('/'),'data_1o_sintomas'] = casos.loc[casos['is'].str.contains('/'),'is'].apply(lambda x: pd.to_datetime(x,format='%d/%m/%Y',errors='raise'))

In [ ]:
casos.loc[casos['is'].str.contains('-'),'data_1o_sintomas'] = casos.loc[casos['is'].str.contains('-'),'is'].apply(lambda x: pd.to_datetime(x,format='%Y-%m-%d',errors='raise'))

In [ ]:
len(casos.loc[casos['data_1o_sintomas'].notna()])

In [ ]:
casos['comunicacao'] = casos['comunicacao'].apply(str)
casos.loc[casos['comunicacao'].str.contains('/'),'data_comunicacao'] = casos.loc[casos['comunicacao'].str.contains('/'),'comunicacao'].apply(lambda x: pd.to_datetime(x,format='%d/%m/%Y',errors='raise'))

In [ ]:
casos.loc[casos['comunicacao'].str.contains('-'),'data_comunicacao'] = casos.loc[casos['comunicacao'].str.contains('-'),'comunicacao'].apply(lambda x: pd.to_datetime(x,format='%Y-%m-%d',errors='raise'))

In [ ]:
casos['is'] = casos['data_1o_sintomas']
casos['comunicacao'] = casos['data_comunicacao']
casos.dtypes

In [ ]:
cc.df = casos